In [1]:
import argparse
from pathlib import Path
from matplotlib import pyplot as plt
from PIL import Image, ImageOps
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
import fnmatch
from face_detection import FaceDetection
from params import RANDOM_SEED, LocationConfig, CreateDataConfig

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/home/kkulawiec/.conda/envs/py39/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."


In [2]:
def load_img(path: Path) -> np.ndarray:
    if not path.exists():
        raise Exception(f"Given file: '{path}' - does not exists!")
    return cv2.imread(str(path))

In [3]:
list_of_face_shapes = [[],[],[]]

In [4]:
def resize(image: np.ndarray) -> np.ndarray:
    img = Image.fromarray(np.array(image).astype(np.uint8))
    img = img.resize((128, 128), Image.ANTIALIAS)
    img = np.array(ImageOps.grayscale(img))
    img = np.expand_dims(img, axis=2)
    return img.astype(np.float32)

In [10]:
def detect_and_crop(face_detect_app: FaceDetection, input_path: Path, output_path: str, img_num: int):
    img = load_img(input_path)
    faces = face_detect_app.detect_face(img)
    if len(faces) == 0 or len(faces) > 1:
        return

    face_box = faces[0].astype(np.int32)
    try:
        new_img = img[face_box[1] : face_box[3], face_box[0] : face_box[2]]
    except IndexError:
        return
    
    if new_img.shape[0]<67 or new_img.shape[1]<49:
        return
    
    list_of_face_shapes[0].append(img_num)
    list_of_face_shapes[1].append(new_img.shape[0])
    list_of_face_shapes[2].append(new_img.shape[1])
    new_img = resize(new_img)
    if new_img is None:
        return
    
    filename = str(input_path).split('/')[-1]
    cv2.imwrite(os.path.join(output_path,filename), new_img)

In [6]:
face_detector = FaceDetection()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


In [7]:
total_f=0
for image_path in tqdm(Path('dataset/wlasne_dane/images/').glob('*.jpg')):
    total_f += 1

18it [00:00, 54629.14it/s]


In [11]:
i=0
for image_path in tqdm(Path('dataset/wlasne_dane/images/').glob('*.jpg'), total=total_f):
    detect_and_crop(face_detector, image_path, 'my_crop_data/', i)
    i+=1

100%|██████████| 18/18 [00:01<00:00, 10.04it/s]


In [9]:
list_of_face_shapes

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 [293,
  1783,
  968,
  1159,
  294,
  145,
  225,
  295,
  180,
  217,
  303,
  257,
  680,
  123,
  169,
  279,
  1890,
  162],
 [204,
  1115,
  685,
  847,
  225,
  105,
  164,
  193,
  128,
  149,
  216,
  186,
  485,
  90,
  129,
  202,
  1200,
  124]]